In [1]:
#SCRAPING ENVOIREMENT

In [2]:
#Importing libaries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [3]:
#Importing page-links
list_url = []
for page in range(1,9,1):
    url = f'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page={page}'
    list_url.append(url)

list_url





['https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=1',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=2',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=3',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=4',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=5',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=6',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=7',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=8']

In [4]:
#Retriving HTMLS
list_htmls =[]
for url in list_url:
    response = requests.get(url, headers={'name': 'Nanna Warming', 'email': 'cnm917@alumni.ku.dk'})
    list_htmls.append(response.text)
    time.sleep(0.5)


    

In [5]:
# Finding links on appartments
links = []
for html in list_htmls:
    soup = BeautifulSoup(html, 'lxml')
    pattern = re.compile(r'href\":\"(/addresses/[^\"]+)')
    matches = pattern.findall(str(soup))

    apartments_list = list(set(matches))


    base_url = "https://www.boligsiden.dk"
    apartment_links = [base_url + link for link in apartments_list] #This list contains appartments
    links.append(apartment_links)

flattened_links = list(itertools.chain(*links))
print(len(flattened_links)) #Counting number of links

400


In [6]:
#Carefull! Runs with X-links at a time
dict = {}
links_med_fejl = []

def fetch_data(link):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
            
        #Finder label hvor afstand er under
        """
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        """
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        #################   NY KODE med skoler    ########################
        #Skolenavn
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1



        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=1) as executor:
    list(tqdm(executor.map(fetch_data, flattened_links[:400]), total=len(flattened_links)))


 20%|████████▏                                 | 78/400 [10:31<43:25,  8.09s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c2-37cb-32b8-e044-0003ba298018 - list index out of range


 28%|███████████▍                             | 111/400 [14:59<38:10,  7.93s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f509d-c94a-32b8-e044-0003ba298018 - list index out of range


 34%|██████████████▏                          | 138/400 [20:13<34:47,  7.97s/it]

...

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50bf-da30-32b8-e044-0003ba298018 - list index out of range


 70%|████████████████████████████▋            | 280/400 [39:32<16:05,  8.05s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/91194325-a12f-4b99-a89f-ea6d471802b8 - list index out of range


 90%|████████████████████████████████████▋    | 358/400 [50:21<05:54,  8.44s/it]


ConnectionError: Could not reach host. Are you offline?

In [11]:
#Denne celle henter data på alle links med fejl fra ovenstående celle! Kør derfor efter ovenstående.
def fetch_data(link):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
    
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        
        #################   NY KODE med skoler    ########################
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=1) as executor:
    list(tqdm(executor.map(fetch_data, links_med_fejl), total=len(links_med_fejl)))


100%|█████████████████████████████████████████████| 5/5 [00:40<00:00,  8.10s/it]


In [13]:
#Converting dict to dataframe:
df_test = pd.DataFrame.from_dict(dict, orient='index')
pd.set_option('display.max_colwidth', None)
df_test



,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Grund,Ejerudgift,Rooms,Build,Toilets,Postal code,link,Elevator,Altan,Terasse
"Jyllandsgade 59,9520",3.100.000 kr.,Skørping Skole,"1,44 km",7.8,3.7,Villa,Energimærke D,Meget lav,"25,33%","26,09‰.",...,Grund: 990 m²,Ejerudg.: 2.239 kr/md,5 værelser,Opført 1935,3 toiletter,9520,https://www.boligsiden.dk/addresses/0a3f50c9-4f92-32b8-e044-0003ba298018,NaN,NaN,NaN
"Mellem Broerne 7C, 2. 1.,9400",3.795.000 kr.,Skansevejens Skole,371 m,7,3.8,Ejerlejlighed,Energimærke C,Lav,"25,4%","26,95‰.",...,NaN,Ejerudg.: 4.211 kr/md,4 værelser,Opført 2007,1 toilet,9400,https://www.boligsiden.dk/addresses/1f7a7527-9eeb-403c-e044-0003ba298018,Elevator: Ja,Altan: Ja,Terrasse: Nej
"Søndre Badevej 6C, 1. 3.,4600",6.495.000 kr.,Sct. Nicolai Skole,"1,14 km",7.5,3.9,Ejerlejlighed,Energimærke A2015,Mellem,"25,26%","21,04‰.",...,NaN,Ejerudg.: 6.153 kr/md,4 værelser,Opført 2018,2 toiletter,4600,https://www.boligsiden.dk/addresses/2cdfa4f0-16be-4af9-8cdb-958f0771f87a,Elevator: Nej,Altan: Nej,Terrasse: Nej
"Ewalds Alle 108,6700",1.798.000 kr.,Rørkjær Skole,737 m,Ikke oplyst,Ikke oplyst,Villa,Energimærke B,Meget lav,"25,8%","25,29‰.",...,Grund: 654 m²,Ejerudg.: 1.507 kr/md,3 værelser,Opført 1960,2 toiletter,6700,https://www.boligsiden.dk/addresses/0a3f50b8-d6f9-32b8-e044-0003ba298018,NaN,NaN,NaN
"Søkrogen 3,2635",5.495.000 kr.,Strandgårdskolen,905 m,5.7,3.7,Villa,Energimærke D,Mellem,25%,25‰.,...,Grund: 734 m²,Ejerudg.: 3.722 kr/md,5 værelser,Opført 1993,2 toiletter,2635,https://www.boligsiden.dk/addresses/0a3f50a7-183b-32b8-e044-0003ba298018,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Nissumgårdsvej 3,8660",9.950.000 kr.,Virring Skole,"4,18 km",7.1,3.7,Landejendom,Energimærke B,Mellem,26%,"24,66‰.",...,Grund: 105012 m²,Ejerudg.: 2.170 kr/md,6 værelser,Opført 1900,2 toiletter,8660,https://www.boligsiden.dk/addresses/0a3f50c2-37cb-32b8-e044-0003ba298018,NaN,NaN,NaN
"Dirchsvej 34, 1.,2300",5.195.000 kr.,Gerbrandskolen,660 m,6.7,3.8,Ejerlejlighed,Energimærke D,Mellem,"23,7%",34‰.,...,NaN,Ejerudg.: 4.054 kr/md,3 værelser,Opført 1943,1 toilet,2300,https://www.boligsiden.dk/addresses/0a3f509d-c94a-32b8-e044-0003ba298018,Elevator: Nej,Altan: Nej,Terrasse: Nej
"Jordrupvej 27,6040",5.395.000 kr.,Vester Nebel Skole,"2,29 km",Ikke oplyst,Ikke oplyst,Landejendom,Energimærke C,Mellem,"25,5%",25‰.,...,Grund: 47762 m²,Ejerudg.: 3.686 kr/md,6 værelser,Opført 1916,1 toilet,6040,https://www.boligsiden.dk/addresses/0a3f50ba-7142-32b8-e044-0003ba298018,NaN,NaN,NaN
"Niels Juels Vej 45,8500",595.000 kr.,Søren Kanne-Skolen,870 m,6.5,3.6,Villa,Energimærke E,Mellem,26%,34‰.,...,Grund: 870 m²,Ejerudg.: 1.084 kr/md,3 værelser,Opført 1937,1 toilet,8500,https://www.boligsiden.dk/addresses/0a3f50bf-da30-32b8-e044-0003ba298018,NaN,NaN,NaN


In [15]:
#Converts to csv-file and save in "Rawdata"
df_test.to_csv('NW_test_file.csv', index=True) #Remember to specify file-name

